Kode pada notebook ini bertujuan untuk menguji dan melakukan prediction request ke model serving:

In [6]:
# Import Library
import requests
import tensorflow as tf
import json
import base64

In [25]:
def create_humor_detection_payload(text):
    feature_dict = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[text.encode("utf-8")])),
    }
    
    example_proto = tf.train.Example(
        features=tf.train.Features(feature=feature_dict)
    ).SerializeToString()
    
    payload = [
        {
            "examples": {
                "b64": base64.b64encode(example_proto).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": payload
    })

def predict(text):
    model_endpoint = "http://localhost:8080/v1/models/humor-detection-model:predict"
    json_payload = create_humor_detection_payload(text)
    
    response = requests.post(model_endpoint, data=json_payload, headers={"Content-Type": "application/json"})
    
    try:
        predictions = response.json().get("predictions")
        if predictions:
            prediction_score = predictions[0][0]
            classification = "Humor detected" if prediction_score > 0.5 else "No humor detected"
        else:
            classification = "Error: No predictions found in response."
    except (ValueError, IndexError, KeyError) as e:
        classification = f"Error processing response: {str(e)}"
    
    return classification

In [28]:
# Contoh teks untuk pengujian
texts = [
    "What's the difference between Donald Trump's hair and a wet racoon?",
    "I told my wife she was drawing her eyebrows too high. She looked surprised.",
    "Why don't scientists trust atoms? Because they make up everything.",
    "I'm reading a book on anti-gravity. It's impossible to put down.",
    "Why did the scarecrow win an award? Because he was outstanding in his field!",
    "This is a sentence"
]

for text in texts:
    pred = predict(text)
    print("Text:", text)
    print('Humor detected:', pred)
    print("\n")

Text: What's the difference between Donald Trump's hair and a wet racoon?
Humor detected: Humor detected


Text: I told my wife she was drawing her eyebrows too high. She looked surprised.
Humor detected: Humor detected


Text: Why don't scientists trust atoms? Because they make up everything.
Humor detected: Humor detected


Text: I'm reading a book on anti-gravity. It's impossible to put down.
Humor detected: Humor detected


Text: Why did the scarecrow win an award? Because he was outstanding in his field!
Humor detected: Humor detected


Text: This is a sentence
Humor detected: No humor detected


